In [390]:
import numpy             as np
import matplotlib.pyplot as plt
import scipy.sparse      as sparse
import scipy.special     as sp
import hylife.utilitis_FEEC.bsplines             as bsp
import hylife.utilitis_FEEC.evaluation           as eva
import hylife.geometry.mappings_analytical       as mapping
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d, Axes3D
import time
%load_ext jupyternotify

The jupyternotify extension is already loaded. To reload it, use:
  %reload_ext jupyternotify


In [391]:
def Parameters(Nel       = [8,8, 2],              # mesh generation on logical domain
               bc        = [True, True, True],      # boundary conditions (True: periodic, False: else)
               p         = [3, 4, 1],         # spline degrees 
               Nq        = [1, 6, 2],
               Lx        = 4.,
               Ly        = 2.,
               Lz        = 1.,
               kind_map  = 1,
               alpha     =0.05):# number of quadrature points per element


    el_b      = [np.linspace(0., 1., Nel + 1) for Nel in Nel]                      # element boundaries
    delta     = [1/Nel for Nel in Nel]                                            # element sizes
    T         = [bsp.make_knots(el_b, p, bc) for el_b, p, bc in zip(el_b, p, bc)]  # knot vectors (for N functions)
    t         = [T[1:-1] for T in T]                                               # reduced knot vectors (for D function)
    NbaseN    = [Nel + p - bc*p for Nel, p, bc in zip(Nel, p, bc)]                 # number of basis functions (N functions)
    NbaseD    = [NbaseN - (1 - bc) for NbaseN, bc in zip(NbaseN, bc)]              #number of basis functions (D functions)

    if kind_map == 1:               #slab geometry
        args_map = [kind_map, [Lx,Ly,Lz]]
    elif kind_map ==3:
        args_map = kind_map, [Lx,Ly,alpha,Lz]

    print('Parameters update Realized')
    return Nel, bc, p, Nq, el_b, delta, T, t, NbaseN, NbaseD, Lx, Ly, Lz, kind_map, args_map, alpha   

In [392]:
def Projectors_ini():
    global T, p, bc, Nq
    import hylife.utilitis_FEEC.projectors_global    as projectors_global
    import hylife.utilitis_FEEC.derivatives          as derivatives
    proj_glob = projectors_global.projectors_3d(T, p, bc,Nq)
    proj_glob.assemble_V3()
    proj_glob.assemble_V2()
    proj_glob.assemble_V1()
    der = derivatives.discrete_derivatives(T, p, bc)
    DIV = der.DIV_3d()
    CUR = der.CURL_3d()
    print('Projectors Initialized')
    return proj_glob, der, DIV, CUR

In [393]:
(Nel, bc, p, Nq, el_b, 
 delta, T, t, NbaseN, 
 NbaseD, Lx, Ly, Lz, kind_map,
 args_map, alpha)=Parameters(Nel=[4,4,2])
(proj_glob, der, DIV, CUR)=Projectors_ini()

Parameters update Realized
Projectors Initialized


In [618]:
B_phys = lambda x,y,z: [np.sin(2*np.pi*x/Lx),y,z]#[(np.sin(2*np.pi*x/Lx)),0,0]

In [619]:
def B_hodge_1(xi1, xi2, xi3): 
    x = [mapping.f(xi1, xi2, xi3, *args_map, j) for j in [1,2,3]] #Analytic Mapping to slab Geometry
    DF_T=np.transpose(DF)
    B_l = DF_T.dot(B_phys(x[0],x[1],x[2]))
    return B_l
def B_ini(xi1,xi2,xi3):
    x = [mapping.f(xi1, xi2, xi3, *args_map, j) for j in [1,2,3]]
    det_df=mapping.det_df(xi1, xi2, xi3, *args_map)
    DF_inv=np.array([[mapping.df_inv(xi1, xi2, xi3, *args_map, (i)*10+(j)) for i in [1,2,3]]
             for j in [1,2,3]])
    B_2=-det_df*DF_inv.dot(B_phys(x[0],x[1],x[2]))
    return B_2

In [623]:
# input, for projector V3:  three 1d point arrays
def B_hodge_x(xi1_arr, xi2, xi3):
    n=xi1_arr.shape[0]
    Bhx=[B_hodge_1(xi1_arr[i1],xi2,xi3)[0] for i1 in range(n)]
    return Bhx
def B_hodge_y(xi1, xi2_arr, xi3):
    n=xi2_arr.shape[0]
    Bhy=[B_hodge_1(xi1,xi2_arr[i2],xi3)[1] for i2 in range(n)]
    return Bhy
def B_hodge_z(xi1, xi2, xi3_arr):
    n=xi3_arr.shape[0]
    Bhy=[B_hodge_1(xi1,xi2,xi3_arr[i3])[1] for i3 in range(n)]
    return Bhy
def B_x(xi1, xi2_arr, xi3_arr):
    n2 = xi2_arr.shape[0]
    n3 = xi3_arr.shape[0]
    Bx_2=np.transpose(np.array([[B_ini(xi1,xi2_arr[i2],xi3_arr[i3])[0]for i2 in range(n2)]
                   for i3 in range(n3)]))
    return Bx_2
def B_y(xi1_arr, xi2, xi3_arr):
    n1 = xi1_arr.shape[0]
    n3 = xi3_arr.shape[0]
    By_2=np.transpose(np.array([[B_ini(xi1_arr[i1],xi2,xi3_arr[i3])[1]for i1 in range(n1)]
                   for i3 in range(n3)]))
    return By_2
def B_z(xi1_arr, xi2_arr, xi3):
    n1 = xi1_arr.shape[0]
    n2 = xi2_arr.shape[0]
    Bz_2=np.transpose(np.array([[B_ini(xi1_arr[i1],xi2_arr[i2],xi3)[2] for i1 in range(n1)]
                   for i2 in range(n2)]))
    return Bz_2

In [624]:
##Project B as a 2 form
Bxh_coeff=proj_glob.PI_21(B_x)
Byh_coeff=proj_glob.PI_22(B_y)
Bzh_coeff=proj_glob.PI_23(B_z)
##Project *B as a 1 form
Bxh_coeff=proj_glob.PI_11(B_hodge_x)
Byh_coeff=proj_glob.PI_12(B_hodge_y)
Bzh_coeff=proj_glob.PI_13(B_hodge_z)

In [625]:
curlB=CUR.dot(np.concatenate((Bxh_coeff.flatten(),Byh_coeff.flatten(),
                              Bzh_coeff.flatten()))).reshape(NbaseN[0],NbaseN[1],NbaseN[2],3)
B_2form=np.array(list(zip(Bx_coeff.flatten(),By_coeff.flatten(),Bz_coeff.flatten())))

#### $DF$, $DF^T$, $DF^{-1}$,$G$, $det(DF)$ in logical coordinates

In [626]:
xi1=el_b[0]
xi2=el_b[1]
xi3=el_b[2]
DF=np.array([[mapping.df(xi1, xi2, xi3, *args_map, (i)*10+(j)) for i in [1,2,3]]
                 for j in [1,2,3]])
DF_inv=np.array([[mapping.df_inv(xi1, xi2, xi3, *args_map, (i)*10+(j)) for i in [1,2,3]]
             for j in [1,2,3]])
det_df=mapping.det_df(xi1, xi2, xi3, *args_map)
DF_T=np.transpose(DF)
G=DF_T.dot(DF)
G_inv=DF_inv.dot(np.transpose(DF_T))
def hodge_op(coeff):
    dim=NbaseN[0]*NbaseN[1]*NbaseN[2]
    coeff_r=coeff.reshape(dim,3)
    B_l = -np.array([-1/(det_df)*G.dot(coeff_r[k]) 
                     for k in range(dim)]).reshape(NbaseN[0],NbaseN[1],NbaseN[2],3)
    return B_l
def hash_op(coeff):
    dim=NbaseN[0]*NbaseN[1]*NbaseN[2]
    coeff_r=coeff.reshape(dim,3)
    B_v = np.array([G_inv.dot(coeff_r[k]) 
                    for k in range(dim)]).reshape(NbaseN[0],NbaseN[1],NbaseN[2],3)
    return B_v

In [627]:
dim=NbaseN[0]*NbaseN[1]*NbaseN[2]
J=hodge_op(curlB)
J1=hash_op(J)
i_JB=np.cross(J1.reshape(dim,3),B_2form).reshape(NbaseN[0],NbaseN[1],NbaseN[2],3)
v=hash_op(i_JB)
vxB=np.cross(v.reshape(dim,3),B_2form).reshape(NbaseN[0],NbaseN[1],NbaseN[2],3)
cur_vxB=CUR.dot(np.concatenate((vxB.reshape(dim,3)[:,0],
                        vxB.reshape(dim,3)[:,1],vxB.reshape(dim,3)[:,2])))

array([-9.24159226e-01,  9.24159226e-01, -1.44624918e+00,  9.72167497e-01,
       -4.50077545e-01,  4.50077545e-01,  2.82048595e+00, -2.34640427e+00,
       -5.80600033e-01,  3.43559193e-01,  4.26073409e-01, -4.26073409e-01,
       -1.16136674e-15,  1.14567260e-15,  5.80600033e-01, -3.43559193e-01,
        1.25553162e-16, -3.13882904e-16,  1.00442529e-15, -9.80884074e-16,
        7.29777751e-16, -6.51307025e-16, -1.96176815e-16,  2.82494613e-16,
        1.09859016e-16, -1.56941452e-16,  1.41247307e-16, -5.49295082e-17,
       -2.58953396e-16,  2.43259250e-16,  2.90341686e-16, -3.13882904e-16,
        0.00000000e+00,  0.00000000e+00, -3.43559193e-01,  5.80600033e-01,
        1.06518352e-01, -3.43559193e-01, -3.43559193e-01,  1.06518352e-01,
        5.80600033e-01, -3.43559193e-01, -3.60965339e-16,  6.90542388e-16,
        1.25553162e-16,  8.63177985e-17,  2.11870960e-16, -9.41648711e-17,
        3.45271194e-16, -2.19718033e-16,  1.72635597e-16, -1.88329742e-16,
       -9.41648711e-17,  